In [1]:
import tensorflow
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import RMSprop

ModuleNotFoundError: No module named 'tensorflow'

In [62]:
train_images = pd.read_csv('../input/train.csv')
test_images = pd.read_csv('../input/test.csv')

In [63]:
y_train = train_images["label"]

X_train = train_images.drop(labels = ["label"],axis = 1) 


In [64]:
from tensorflow.keras import backend as K

  
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
X_train /= 255
test_images /= 255

In [66]:
X_train = X_train.values.reshape(-1,28,28,1)
test_images = test_images.values.reshape(-1,28,28,1)

In [60]:
X_train.shape

(42000, 28, 28, 1)

In [68]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.1, random_state=101)

In [67]:
y_train = tensorflow.keras.utils.to_categorical(y_train, 10)

In [72]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28,28,1)))
# 64 3x3 kernels
model.add(Conv2D(64, (3, 3), activation='relu'))
# Reduce by taking the max of each 2x2 block
model.add(MaxPooling2D(pool_size=(2, 2)))
# Dropout to avoid overfitting
model.add(Dropout(0.25))
# Flatten the results to one dimension for passing into our final layer
model.add(Flatten())
# A hidden layer to learn with
model.add(Dense(128, activation='relu'))
# Another dropout
model.add(Dropout(0.5))
# Final categorization from 0-9 with softmax
model.add(Dense(10, activation='softmax'))

In [73]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)              

In [74]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [75]:
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=10,
                    verbose=2,
                    validation_data=(X_test, y_test))

Train on 37800 samples, validate on 4200 samples
Epoch 1/10
37800/37800 - 59s - loss: 0.2548 - acc: 0.9231 - val_loss: 0.0601 - val_acc: 0.9810
Epoch 2/10
37800/37800 - 59s - loss: 0.1009 - acc: 0.9696 - val_loss: 0.0480 - val_acc: 0.9869
Epoch 3/10
37800/37800 - 59s - loss: 0.0728 - acc: 0.9778 - val_loss: 0.0377 - val_acc: 0.9888
Epoch 4/10
37800/37800 - 58s - loss: 0.0585 - acc: 0.9811 - val_loss: 0.0393 - val_acc: 0.9893
Epoch 5/10
37800/37800 - 59s - loss: 0.0516 - acc: 0.9840 - val_loss: 0.0381 - val_acc: 0.9893
Epoch 6/10
37800/37800 - 59s - loss: 0.0457 - acc: 0.9858 - val_loss: 0.0337 - val_acc: 0.9902
Epoch 7/10
37800/37800 - 59s - loss: 0.0388 - acc: 0.9879 - val_loss: 0.0372 - val_acc: 0.9910
Epoch 8/10
37800/37800 - 59s - loss: 0.0359 - acc: 0.9892 - val_loss: 0.0402 - val_acc: 0.9888
Epoch 9/10
37800/37800 - 59s - loss: 0.0321 - acc: 0.9899 - val_loss: 0.0417 - val_acc: 0.9888
Epoch 10/10
37800/37800 - 59s - loss: 0.0296 - acc: 0.9907 - val_loss: 0.0422 - val_acc: 0.9907


Was it worth the wait?

In [76]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.042247241042527355
Test accuracy: 0.9907143


In [78]:
results = model.predict(test_images)

results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [80]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


In [81]:
submission.to_csv("submission_mnist.csv",index=False)